# Decision Trees

### Coding a decision tree binary classifier from scratch

Will run through the algorithm implementation first on a small toy dataset, predicting whether someone will play tennis based on the weather conditions. Next, we will see if it can work on a real life dataset predicting Covid-19 cases. 

## Import Libraries and handle weather data

For each function/computation, I have written example code underneath for testing and demonstration purposes.

In [1]:
import pandas as pd
import math
from pprint import pprint

In [2]:
# Importing the weather data
weather_df = pd.read_csv('weather-data.csv')
weather_df.drop("Day", 'columns', inplace=True)

#preview the data
weather_df

<ipython-input-2-a6381cc0b03a>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  weather_df.drop("Day", 'columns', inplace=True)


,Outlook,Temperature,Humidity,Wind,Decision
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rainfall,Mild,High,Weak,Yes
4,Rainfall,Cool,Normal,Weak,Yes
5,Rainfall,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rainfall,Mild,Normal,Weak,Yes


In [3]:
weather_df[(weather_df["Outlook"] == "Sunny") & (weather_df["Humidity"] == "High")]


,Outlook,Temperature,Humidity,Wind,Decision
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
7,Sunny,Mild,High,Weak,No


In [4]:
# Extract feature columns and the decision column

weather_features = [_ for _ in weather_df]
weather_decision = weather_features.pop(-1)
print(weather_features)
print(weather_decision)

['Outlook', 'Temperature', 'Humidity', 'Wind']
Decision


## Helper functions

In [5]:
def calculate_entropy(counts):
    """counts lists of the number of occurences of each decision.
    From this the probability can be calculated and then the entropy."""
    
    entropy = 0
    for count in counts:
        if count == 0:
            continue
        p = count/sum(counts)
        entropy += p*math.log(p, 2)
    entropy *= -1
    return entropy

# Testing:
SS_E = calculate_entropy(weather_df.groupby([weather_decision]).size())
print(f"E(N) = {SS_E}")

outlook = weather_df.groupby(["Outlook", "Decision"]).size()

sun = calculate_entropy(outlook["Sunny"])
rain = calculate_entropy(outlook["Rainfall"])
over = calculate_entropy(outlook["Overcast"])

print(f"E(Sunny): {sun}\nE(Rain): {rain}\nE(Overcast): {over}")

E(N) = 0.9402859586706309
E(Sunny): 0.9709505944546686
E(Rain): 0.9709505944546686
E(Overcast): -0.0


In [6]:
def information_gain(n_rows, samplespace_E, feature_E):
    """n_rows is the number of rows in the dataset, samplespace_E = E(N),
    and feature_E is a dictionary consisting of feature values as keys
    and a tuple (occurences, entropy) as the values. E.g for the feature outlook
    the dictionary of entropies would be {"Sunny": (5, 0.97), "Rainfall: (5, 0.97), "Overcast": (4, 0)}"""
    
    info_gain = samplespace_E
    
    for value in feature_E.values():
        info_gain -= (value[0]/n_rows) * value[1]
        
    return info_gain

# Testing
outlook_Es = {"Sunny": (5, sun), "Rainfall": (5, rain), "Overcast": (4, over)}

print(f"IG(Outlook) = {information_gain(14, SS_E, outlook_Es)}")

IG(Outlook) = 0.2467498197744391


In [7]:
# Tests purity of the df

def pure(df, decision_col):
    
    if len(df[decision_col].unique()) == 1:
        return True
    
    else:
        return False

In [8]:
# Classifies the df based on the majority decision

def classify_data(df, label_col):
    
    majority_decision = df[label_col].mode()[0]
    
    count_for_majority = (df[label_col] == majority_decision).sum()
    total_examples = len(df)
    probability = count_for_majority/total_examples
    
    return (majority_decision, probability)

sun_humid = weather_df[(weather_df["Outlook"] == "Sunny") & (weather_df["Humidity"] == "High")]
print(sun_humid)
classify_data(weather_df, "Decision")
classify_data(sun_humid, "Decision")

  Outlook Temperature Humidity    Wind Decision
0   Sunny         Hot     High    Weak       No
1   Sunny         Hot     High  Strong       No
7   Sunny        Mild     High    Weak       No


('No', 1.0)

## Splitting the data

In [10]:
def split_data(df, split_feature, split_value):
    """Input the current df, and the feature and value to split the df on.
    Returns df which satisfies the specified feature value"""
    
    yes_df = df[df[split_feature] == split_value]

    return yes_df


sunny_df = split_data(weather_df, "Outlook", "Sunny")
print(sunny_df)

   Outlook Temperature Humidity    Wind Decision
0    Sunny         Hot     High    Weak       No
1    Sunny         Hot     High  Strong       No
7    Sunny        Mild     High    Weak       No
8    Sunny        Cool   Normal    Weak      Yes
10   Sunny        Mild   Normal  Strong      Yes


In [11]:
# Finds the most informative feature to split the data on next

def find_next_split(df, feature_cols, label_col):
    """Returns the feature from feature_cols which has the highest information gain for the label_col."""
    
    samplespace_e = calculate_entropy(df.groupby([label_col]).size())
    total_entries = len(df)
    
    # initialize variables
    info_gain = {}
    feature_values = {} #dict of all the unique values each feature has
    
    
    for feature in df:
        feature_values[feature] = df[feature].unique()
    
    
    for feature in feature_cols:
        entropies = {}
        feature_counts = df.groupby([feature, label_col]).size()
        
        
        for value in feature_values[feature]:
            value_counts = feature_counts[value]
            entropies[value] = [sum(value_counts), calculate_entropy(value_counts)]
            
            
        info_gain[feature] = information_gain(total_entries, samplespace_e, entropies)
    next_node = max(info_gain, key = info_gain.get)
        
    return next_node


#Testing                             
initial_split = find_next_split(weather_df, ["Outlook", "Temperature", "Humidity", "Wind"], "Decision")

sunny_split = find_next_split(sunny_df, ["Temperature", "Humidity", "Wind"], "Decision")

print(f"Initial split: {initial_split}\nSplit from Outlook = Sunny: {sunny_split}")

Initial split: Outlook
Split from Outlook = Sunny: Humidity


## Algorithm

In [12]:
# Russel and Norvig, 2021 Learn Decision Tree Algorithm on p678.

def decision_tree(df, feature_cols, decision_col, counter=0, max_depth = 5):
    
    """df is the data remaining which needs to be split,
    feature_cols is a list of the predictor variables, decision_col is the response variable,
    the counter counts tree depth, and max_depth limits the tree depth."""
    
    
    if (pure(df, decision_col)) or (counter == max_depth):
        classification = classify_data(df, decision_col)
        return classification
    
    else:
        counter += 1
        
        next_feature = find_next_split(df, feature_cols, decision_col)
        
        tree = {}
        values = df[next_feature].unique()
        
        possible_features = feature_cols.copy()
        possible_features.remove(next_feature)
        
        current_node = next_feature
        
        for value in values: # recursion
            value_df = split_data(df, next_feature, value)
            subtree = decision_tree(value_df, possible_features, decision_col, counter, max_depth)
            tree[current_node, value] = subtree
        
        return tree

    
# Final tree for weather data:
weather_tree = decision_tree(weather_df, weather_features, weather_decision)
pprint(weather_tree)    

{('Outlook', 'Overcast'): ('Yes', 1.0),
 ('Outlook', 'Rainfall'): {('Wind', 'Strong'): ('No', 1.0),
                           ('Wind', 'Weak'): ('Yes', 1.0)},
 ('Outlook', 'Sunny'): {('Humidity', 'High'): ('No', 1.0),
                        ('Humidity', 'Normal'): ('Yes', 1.0)}}


In [13]:
def classify_example(example, df, tree, decision_col):
    """example is an individual row from a df, tree is a fully grown DT
    and decisions is a tuple of possible output decisions.
    The function traverses the tree to output the leaf node corresponding to the example."""

    decisions = df[decision_col].unique()
    
    current_node = list(tree.keys())[0][0] # instantiate the current position in the tree as the first node e.g Outlook
    
    residual_tree = tree.copy()
    
    residual_df = df.copy()
    
    while current_node not in decisions:
        
        node_value = example[current_node]
        
        residual_df = residual_df[residual_df[current_node] == node_value]
        
        try:
            residual_tree = residual_tree[current_node, node_value]
        except: # no entries in the tree with this value. Therefore classify based on probability.
            current_node = classify_data(residual_df, decision_col)[0]
        
        else:
            if type(residual_tree) == dict:
                current_node = list(residual_tree.keys())[0][0]

            else: # reached leaf node
                current_node = residual_tree[0]
    
    return current_node

In [14]:
example = weather_df.iloc[3]
print(example)
classify_example(example, weather_df, weather_tree, "Decision")

Outlook        Rainfall
Temperature        Mild
Humidity           High
Wind               Weak
Decision            Yes
Name: 3, dtype: object


'Yes'

In [15]:
def calculate_accuracy(df, tree, decision_col):
    """Calculates the accuracy of my DT classification using test data."""
    
    df["Classification"] = df.apply(classify_example, axis = 1, args = (df, tree, decision_col))
    
    df["Correct"] = df[decision_col] == df["Classification"]
    
    accuracy = df["Correct"].mean()
    return accuracy

calculate_accuracy(weather_df, weather_tree, "Decision") # accuracy = 1 because it is the training data

1.0

In [16]:
weather_df

,Outlook,Temperature,Humidity,Wind,Decision,Classification,Correct
0,Sunny,Hot,High,Weak,No,No,True
1,Sunny,Hot,High,Strong,No,No,True
2,Overcast,Hot,High,Weak,Yes,Yes,True
3,Rainfall,Mild,High,Weak,Yes,Yes,True
4,Rainfall,Cool,Normal,Weak,Yes,Yes,True
5,Rainfall,Cool,Normal,Strong,No,No,True
6,Overcast,Cool,Normal,Strong,Yes,Yes,True
7,Sunny,Mild,High,Weak,No,No,True
8,Sunny,Cool,Normal,Weak,Yes,Yes,True
9,Rainfall,Mild,Normal,Weak,Yes,Yes,True


## Additional Dataset: Predicting Covid-19 from symptoms and other variables

### Preparing the data

In [17]:
covid_df = pd.read_csv("covid-data.csv")
covid_df

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,Yes,Yes,Yes,Yes,Yes,No,No,No,No,Yes,...,Yes,Yes,No,Yes,No,Yes,Yes,No,No,Yes
1,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,No,No,...,Yes,No,No,No,Yes,Yes,No,No,No,Yes
2,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,...,Yes,Yes,Yes,No,No,No,No,No,No,Yes
3,Yes,Yes,Yes,No,No,Yes,No,No,Yes,Yes,...,No,No,Yes,No,Yes,Yes,No,No,No,Yes
4,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,...,No,Yes,No,Yes,No,Yes,No,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,No,...,Yes,Yes,No,No,No,No,No,No,No,Yes
5430,Yes,Yes,Yes,No,Yes,Yes,No,Yes,No,Yes,...,Yes,No,No,No,No,No,No,No,No,Yes
5431,Yes,Yes,Yes,No,No,No,No,No,Yes,No,...,No,No,No,No,No,No,No,No,No,No
5432,Yes,Yes,Yes,No,Yes,No,No,Yes,Yes,No,...,No,No,No,No,No,No,No,No,No,No


In [18]:
# Inspecting the data

overall_numbers = covid_df.groupby(["COVID-19"]).size()
pos = overall_numbers["Yes"]
neg = overall_numbers["No"]
print(f"Positive cases: {pos}\nNegative cases: {neg}\n{(pos/(pos+neg))*100}% positive")

Positive cases: 4383
Negative cases: 1051
80.65881486934119% positive


In [20]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(covid_df, train_size = 0.8, random_state=1)

In [21]:
features = [_ for _ in covid_df]
features.remove("COVID-19")
label = "COVID-19"
train
covid_tree = decision_tree(train, features, label, max_depth = 5)

In [22]:
pprint(covid_tree)

{('Abroad travel', 'No'): {('Sore throat', 'No'): {('Breathing Problem', 'No'): {('Attended Large Gathering', 'No'): ('No',
                                                                                                                      1.0),
                                                                                 ('Attended Large Gathering', 'Yes'): {('Dry Cough', 'No'): ('No',
                                                                                                                                             0.9803921568627451),
                                                                                                                       ('Dry Cough', 'Yes'): ('Yes',
                                                                                                                                              1.0)}},
                                                   ('Breathing Problem', 'Yes'): {('Attended Large Gathering', 'No'): {('Dry Cough', 'No'): ('No

In [23]:
calculate_accuracy(test, covid_tree, label)

<ipython-input-15-d7572a0f6807>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Classification"] = df.apply(classify_example, axis = 1, args = (df, tree, decision_col))
<ipython-input-15-d7572a0f6807>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Correct"] = df[decision_col] == df["Classification"]


0.9494020239190433

In [24]:
# testing different tree depths

max_depth_acc = {}
for i in range(1, 10):
    t = decision_tree(train, features, label, max_depth = i)
    max_depth_acc[i] = calculate_accuracy(test, t, label)
max_depth_acc

<ipython-input-15-d7572a0f6807>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Classification"] = df.apply(classify_example, axis = 1, args = (df, tree, decision_col))
<ipython-input-15-d7572a0f6807>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Correct"] = df[decision_col] == df["Classification"]


{1: 0.8022079116835327,
 2: 0.8988040478380864,
 3: 0.921803127874885,
 4: 0.9199632014719411,
 5: 0.9494020239190433,
 6: 0.9622815087396505,
 7: 0.9751609935602575,
 8: 0.9797608095676172,
 9: 0.9806807727690893}

### Let's compare the trees with a maximum depth of 3 to 5

In [25]:
t3 = decision_tree(train, features, label, max_depth=3)
pprint(t3)

{('Abroad travel', 'No'): {('Sore throat', 'No'): {('Breathing Problem', 'No'): ('No',
                                                                                 0.9211009174311927),
                                                   ('Breathing Problem', 'Yes'): ('Yes',
                                                                                  0.6521739130434783)},
                           ('Sore throat', 'Yes'): {('Attended Large Gathering', 'No'): ('Yes',
                                                                                         0.696969696969697),
                                                    ('Attended Large Gathering', 'Yes'): ('Yes',
                                                                                          1.0)}},
 ('Abroad travel', 'Yes'): ('Yes', 1.0)}


This would benefit from being pruned. Both leaves from the 'Sore throat' question are 'Yes' so it is pointless.

In [26]:
t5 = decision_tree(train, features, label, max_depth=5)
pprint(t5)

{('Abroad travel', 'No'): {('Sore throat', 'No'): {('Breathing Problem', 'No'): {('Attended Large Gathering', 'No'): ('No',
                                                                                                                      1.0),
                                                                                 ('Attended Large Gathering', 'Yes'): {('Dry Cough', 'No'): ('No',
                                                                                                                                             0.9803921568627451),
                                                                                                                       ('Dry Cough', 'Yes'): ('Yes',
                                                                                                                                              1.0)}},
                                                   ('Breathing Problem', 'Yes'): {('Attended Large Gathering', 'No'): {('Dry Cough', 'No'): ('No

### Discussion of the final depth 5 tree

Despite the difficulty of reading the depth 5 tree, there are still some key takeaways. In general, the algorithm is less certain when all answers are 'no' for the lifestyle/contact factors, as opposed to the symptom factors. The tree chooses nodes like “Abroad travel”, “contact with Covid patient” and “Attended large gathering” early on and these can strongly indicate a positive Covid case. There is still relatively low confidence at some of the leaf nodes though, which would improve with a deeper tree. This classifier would most likely benefit from expanding the tree further and then back-pruning the tree so as to simplify it and ensure it does not suffer from overfitting. The Covid dataset is well suited to a decision tree classification, mirroring the role of a doctor who asks questions about symptoms to diagnose a patient.